# Generate the data in this package

Evaluate the integrals in (22) and (24) of [arXiv:2006.05292] for a range of values of `n_average`.

In [ ]:
%pylab inline

import pandas as pd
from tqdm import tqdm

from median_marginalized.utils import (
    likelihood_integrand, whitening_integrand, recursively_integrate
)

In [ ]:
mpl.rcParams["font.family"] = "serif"
mpl.rcParams["font.serif"] = "Computer Modern Roman"
mpl.rcParams["font.size"] = 20
mpl.rcParams["text.usetex"] = True
mpl.rcParams["grid.alpha"] = 0
mpl.rcParams['text.latex.preamble'] = r'\newcommand{\mathdefault}[1][]{}'

In [ ]:
n_averages = np.arange(1, 128, 2)

all_whitening = pd.DataFrame()
all_likelihood = pd.DataFrame()

_x_values = np.concatenate([np.array([0]), np.logspace(-5, 2, 10000)])

all_whitening["x"] = _x_values
all_likelihood["x"] = _x_values ** 2

for integrand, label in zip(
    [likelihood_integrand, whitening_integrand], ["likelihood", "whitening"]
):

    _y_values = np.log(np.array([
        np.array([
            recursively_integrate(integrand, xx, int(n_average), 1)
            for n_average in n_averages
        ])
        for xx in tqdm(_x_values)
    ])).T

    for y_values, n_average in zip(_y_values, n_averages):
        if label == "likelihood":
            y_values -= np.log(np.trapz(np.exp(y_values), _x_values ** 2))
            all_likelihood[n_average] = y_values
        else:
            y_values -= np.log(np.trapz(np.exp(y_values), _x_values)) + np.log(2)
            all_whitening[n_average] = y_values

    for ii in range(len(_y_values)):
        plt.plot(_x_values ** 2, _y_values[ii], color=f"C{ii}")
    ax = plt.gca()
    ax.ticklabel_format(style='plain', useOffset=False)
    plt.xlim(0, max(_x_values ** 2))
    plt.show()
    plt.close()

    for ii in range(len(_y_values)):
        plt.plot(_x_values ** 2, _y_values[ii], color=f"C{ii}")
    plt.xlim(0, 20)
    plt.ylim(-10, 0)
    ax = plt.gca()
    ax.ticklabel_format(style='plain', useOffset=False)
    plt.show()
    plt.close()

all_whitening.to_csv("whitening.dat", sep="\t", index=False)
all_likelihood.to_csv("likelihood.dat", sep="\t", index=False)